In [63]:
import pandas as pd
from src.dimension_modeler import Dimension_Modeler
from src.util import update_start_date

### DIM PROMOTIONS
- promotion_id (int)
- promotion_name (str) DONE
- discount_percentage (decimal 5,2) DONE
- description (str) DONE

In [91]:
promo_df = pd.read_csv('datasets/donut_shop_promos.csv')

In [94]:
promo_df.head()

,Promo Name,Discount Perc,Description
0,BOGO,50%,"Happens sporadically throughout the year, all ..."
1,WINTER SPECIAL,50%,"During Dec, Jan, Feb, all products 50% off."
2,BACK TO SCHOOL,25%,"During first week of September, 25% off all pr..."


#### Conversion of discount percentage to float

In [96]:
promo_df['Discount Perc'] = promo_df['Discount Perc'].apply(lambda x: x.split('%')[0])

In [100]:
promo_df['Discount Perc'] = promo_df['Discount Perc'].astype(float)

In [101]:
promo_df.dtypes

Promo Name        object
Discount Perc    float64
Description       object
dtype: object

In [103]:
promo_df['Discount Perc'] = promo_df['Discount Perc']/100

In [104]:
promo_df

,Promo Name,Discount Perc,Description
0,BOGO,0.50,"Happens sporadically throughout the year, all ..."
1,WINTER SPECIAL,0.50,"During Dec, Jan, Feb, all products 50% off."
2,BACK TO SCHOOL,0.25,"During first week of September, 25% off all pr..."


#### Adding id column, rearranging and fixing column names and saving to csv

In [105]:
promo_dim_model = Dimension_Modeler(promo_df)

In [106]:
promo_dim_model.make_id_col('promotion_id')

,promotion_id,Promo Name,Discount Perc,Description
0,1,BOGO,0.50,"Happens sporadically throughout the year, all ..."
1,2,WINTER SPECIAL,0.50,"During Dec, Jan, Feb, all products 50% off."
2,3,BACK TO SCHOOL,0.25,"During first week of September, 25% off all pr..."


In [107]:
promo_dim_model.df

,promotion_id,Promo Name,Discount Perc,Description
0,1,BOGO,0.50,"Happens sporadically throughout the year, all ..."
1,2,WINTER SPECIAL,0.50,"During Dec, Jan, Feb, all products 50% off."
2,3,BACK TO SCHOOL,0.25,"During first week of September, 25% off all pr..."


In [108]:
promo_dim_model.df.columns = promo_dim_model.df.columns.str.lower().str.replace(' ', '_')

In [111]:
promo_dim_model.df.rename(columns={'promo_name': 'promotion_name', 'discount_perc': 'discount_percentage'}, inplace=True)

In [114]:
promo_dim_model.df.dtypes

promotion_id             int64
promotion_name          object
discount_percentage    float64
description             object
dtype: object

In [115]:
promo_dim_model.df

,promotion_id,promotion_name,discount_percentage,description
0,1,BOGO,0.50,"Happens sporadically throughout the year, all ..."
1,2,WINTER SPECIAL,0.50,"During Dec, Jan, Feb, all products 50% off."
2,3,BACK TO SCHOOL,0.25,"During first week of September, 25% off all pr..."


In [116]:
promo_dim_model.make_csv('dim_promotions')

dim_promotions.csv created in datasets/dimensions/


### DIM PAYMENT METHODS
- payment_method_id (int) DONE
- payment_method (str) DONE

In [65]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [66]:
payment_method_df = transactions_df[['Payment Method']]
payment_method_df

,Payment Method
0,Cash
1,Cash
2,Cash
3,Card
4,Card
...,...
756130,Card
756131,Card
756132,Cash
756133,Cash


In [79]:
payment_method_df =payment_method_df.drop_duplicates(keep='first')

In [83]:
payment_method_df.drop(columns='level_0', inplace=True)

In [85]:
payment_method_df.columns = payment_method_df.columns.str.lower().str.replace(' ', '_')

In [86]:
payment_method_df

,payment_method
0,Cash
1,Card


In [87]:
payment_method_dim_model = Dimension_Modeler(payment_method_df)

In [88]:
payment_method_dim_model.make_id_col('payment_method_id')
payment_method_dim_model.df

,payment_method_id,payment_method
0,1,Cash
1,2,Card


In [89]:
payment_method_dim_model.df.dtypes

payment_method_id     int64
payment_method       object
dtype: object

In [90]:
payment_method_dim_model.make_csv('dim_payment_methods')

dim_payment_methods.csv created in datasets/dimensions/


### DIM PRODUCTS
- product_id (int) (Natural Key) DONE
- product_key (int) (Surrogate Key) DONE
- product_name (str) DONE
- product_category (str) DONE
- product_price (decimal 10,2) DONE
- product_cost (decimal 10,2) DONE 
- is_current(boolean) DONE
- start_date (date) DONE 
- end_date (date) DONE

In [2]:
products_df = pd.read_csv('datasets/donut_shop_product_info.csv')

In [3]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


#### Getting historical prices and dates associated

In [4]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [5]:
price_history = []


In [6]:
for x in transactions_df.groupby('Category')['Price per Unit'].value_counts().index:
    price_history.append({
            'category': x[0],
            'price': x[1],
            'start_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].min(),
            'end_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].max(),
            }
                         )

In [7]:
price_history

[{'category': 'Donut',
  'price': 1.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Donut',
  'price': 0.5,
  'start_date': '2015-01-01 01:35:44',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Drinks',
  'price': 2.0,
  'start_date': '2020-01-01 03:05:09',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Drinks',
  'price': 1.25,
  'start_date': '2015-01-01 00:56:32',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Hot Foods',
  'price': 3.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Hot Foods',
  'price': 1.5,
  'start_date': '2015-01-01 01:41:45',
  'end_date': '2019-12-31 23:32:46'}]

In [8]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].max()

'2024-12-31 23:53:27'

In [9]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].min()

'2020-01-01 00:27:34'

In [10]:
dim_prod_df = pd.DataFrame(price_history)
    

#### Getting is_current field

In [11]:
dim_prod_df

,category,price,start_date,end_date
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46


In [12]:
dim_prod_df['max_start_date'] = dim_prod_df.groupby('category')['start_date'].transform('max')
dim_prod_df

,category,price,start_date,end_date,max_start_date
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,2020-01-01 00:27:34
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,2020-01-01 03:05:09
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,2020-01-01 03:05:09
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,2020-01-01 00:27:34


In [13]:
dim_prod_df['is_current'] = dim_prod_df['max_start_date'] == dim_prod_df['start_date']
dim_prod_df

,category,price,start_date,end_date,max_start_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,2020-01-01 00:27:34,False
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,2020-01-01 03:05:09,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,2020-01-01 03:05:09,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,2020-01-01 00:27:34,False


In [14]:
dim_prod_df.drop(columns='max_start_date', inplace=True)


#### Updating end_date

In [15]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [16]:
max_date = pd.Timestamp.max.strftime('%Y-%m-%d %H:%M:%S')
max_date

'2262-04-11 23:47:16'

In [17]:
dim_prod_df.loc[dim_prod_df['is_current'], 'end_date'] = max_date

#### Updating Start_date

In [18]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2020-01-01 03:05:09,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [19]:
dim_prod_df = dim_prod_df.groupby('category', group_keys=False).apply(update_start_date)

In [20]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


#### Adding Historical prices


In [21]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [22]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


In [23]:
dim_prod_df = pd.merge(dim_prod_df, products_df, how='left', left_on='category', right_on='Product Category')

In [24]:
dim_prod_df = dim_prod_df[['category', 'price', 'start_date', 'end_date', 'is_current', 'Product Cost', 'Historical Cost']]

In [25]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost,Historical Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,0.1
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,0.1
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15,1.0
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.15,1.0
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80,0.5
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.80,0.5


In [26]:
dim_prod_df.loc[~dim_prod_df['is_current'], 'Product Cost'] = dim_prod_df['Historical Cost']
dim_prod_df.drop(columns='Historical Cost', inplace=True)

In [27]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50


In [28]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


#### Adding product name

In [29]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50


In [30]:
products_list_df = transactions_df[['Product Name', 'Category']].drop_duplicates(keep='first').reset_index().drop(columns='index')

In [31]:
products_list_df

,Product Name,Category
0,Coke,Drinks
1,Blueberry,Donut
2,Cinnamon Swirl,Donut
3,Croissant Hot Dog,Hot Foods
4,Biscuits,Hot Foods
5,Sprite,Drinks
6,Blueberry Donut Hole,Donut
7,Glaze,Donut
8,Chocolate,Donut
9,Tropicana,Drinks


In [32]:
dim_prod_df = pd.merge(dim_prod_df, products_list_df, how='left', left_on='category', right_on='Category')


In [33]:
dim_prod_df.drop(columns='Category', inplace=True)
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost,Product Name
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry
1,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Cinnamon Swirl
2,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry Donut Hole
3,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Glaze
4,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Chocolate
5,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Chocolate Donut Hole
6,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Frosted Donut
7,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Donut Odyssey
8,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Frosted Donut Sprinkle
9,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Boston Creme


#### Adding surrogate key / PK

In [34]:
prod_dim_model = Dimension_Modeler(dim_prod_df)

In [35]:
prod_dim_model.make_id_col('product_key')

,product_key,category,price,start_date,end_date,is_current,Product Cost,Product Name
0,1,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry
1,2,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Cinnamon Swirl
2,3,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry Donut Hole
3,4,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Glaze
4,5,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Chocolate
5,6,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Chocolate Donut Hole
6,7,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Frosted Donut
7,8,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Donut Odyssey
8,9,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Frosted Donut Sprinkle
9,10,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Boston Creme


#### Product ID creation (Natural key/business key)

In [36]:
dim_prod_df = prod_dim_model.df

In [37]:
dim_prod_df.head()

,product_key,category,price,start_date,end_date,is_current,Product Cost,Product Name
0,1,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry
1,2,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Cinnamon Swirl
2,3,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry Donut Hole
3,4,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Glaze
4,5,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Chocolate


In [38]:
grouped_df = dim_prod_df.groupby('Product Name', group_keys=False)

modified_groups = []

In [39]:
id_counter = 100

for group_name, group_data in grouped_df:
    current_df = grouped_df.get_group(group_name).copy()
    current_df['product_id'] = id_counter
    id_counter += 1
    modified_groups.append(current_df)

combined_df = pd.concat(modified_groups)

In [41]:
dim_prod_df = combined_df

In [45]:
dim_prod_df.sort_values(by='product_id')


,product_key,category,price,start_date,end_date,is_current,Product Cost,Product Name,product_id
45,46,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80,Biscuits,100
50,51,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50,Biscuits,100
0,1,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry,101
14,15,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Blueberry,101
2,3,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry Donut Hole,102
16,17,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Blueberry Donut Hole,102
9,10,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Boston Creme,103
23,24,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Boston Creme,103
4,5,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Chocolate,104
18,19,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Chocolate,104


#### Fixing any dtypes and reordering columns & saving dim_products csv file.

In [46]:
dim_prod_df.head()

,product_key,category,price,start_date,end_date,is_current,Product Cost,Product Name,product_id
45,46,Hot Foods,3.0,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.8,Biscuits,100
50,51,Hot Foods,1.5,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.5,Biscuits,100
0,1,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry,101
14,15,Donut,0.5,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.1,Blueberry,101
2,3,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry Donut Hole,102


In [51]:
dim_prod_df.dtypes

product_key       int64
category         object
price           float64
start_date       object
end_date         object
is_current         bool
Product Cost    float64
Product Name     object
product_id        int64
dtype: object

In [54]:
dim_prod_df['start_date'] = pd.to_datetime(dim_prod_df['start_date'])
dim_prod_df['end_date'] = pd.to_datetime(dim_prod_df['end_date'])

In [56]:
dim_prod_df.head()

,product_key,category,price,start_date,end_date,is_current,Product Cost,Product Name,product_id
45,46,Hot Foods,3.0,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.8,Biscuits,100
50,51,Hot Foods,1.5,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.5,Biscuits,100
0,1,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry,101
14,15,Donut,0.5,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.1,Blueberry,101
2,3,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry Donut Hole,102


In [57]:
dim_prod_df.columns = dim_prod_df.columns.str.lower().str.replace(' ', '_')
dim_prod_df.head()

,product_key,category,price,start_date,end_date,is_current,product_cost,product_name,product_id
45,46,Hot Foods,3.0,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.8,Biscuits,100
50,51,Hot Foods,1.5,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.5,Biscuits,100
0,1,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry,101
14,15,Donut,0.5,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.1,Blueberry,101
2,3,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry Donut Hole,102


In [58]:
dim_prod_df.rename(columns={'category': 'product_category', 'price': 'product_price'}, inplace=True)

In [59]:
dim_prod_df.head()

,product_key,product_category,product_price,start_date,end_date,is_current,product_cost,product_name,product_id
45,46,Hot Foods,3.0,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.8,Biscuits,100
50,51,Hot Foods,1.5,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.5,Biscuits,100
0,1,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry,101
14,15,Donut,0.5,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.1,Blueberry,101
2,3,Donut,1.0,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.1,Blueberry Donut Hole,102


In [60]:
prod_dim_model.df = dim_prod_df[['product_key', 'product_id', 'product_name', 'product_category', 'product_price', 'product_cost', 'start_date', 'end_date', 'is_current']]

In [61]:
prod_dim_model.df.head()

,product_key,product_id,product_name,product_category,product_price,product_cost,start_date,end_date,is_current
45,46,100,Biscuits,Hot Foods,3.0,0.8,2019-12-31 23:32:46,2262-04-11 23:47:16,True
50,51,100,Biscuits,Hot Foods,1.5,0.5,2015-01-01 01:41:45,2019-12-31 23:32:46,False
0,1,101,Blueberry,Donut,1.0,0.1,2019-12-31 23:49:08,2262-04-11 23:47:16,True
14,15,101,Blueberry,Donut,0.5,0.1,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,3,102,Blueberry Donut Hole,Donut,1.0,0.1,2019-12-31 23:49:08,2262-04-11 23:47:16,True


In [62]:
prod_dim_model.make_csv('dim_products')

dim_products.csv created in datasets/dimensions/


### DIM EMPLOYEES
- employee_id (int) DONE 
- employee_name (str) DONE 
- job_title (str) DONE 

In [ ]:
employee_df = pd.read_csv('datasets/donut_shop_employees.csv')


In [ ]:
dim_model = Dimension_Modeler(employee_df)

In [ ]:
employee_df.columns = ['employee_name', 'job_title']

In [ ]:
employee_df.dtypes

In [ ]:
employee_df = dim_model.make_id_col('employee_id')

In [ ]:
employee_df.dtypes

In [ ]:
employee_df

In [ ]:
dim_model.make_csv('dim_employees')